In [1]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.spatial import distance_matrix

In [2]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py39_cu102_pyt1101/download.html


In [3]:
num_vertices = 100

In [4]:
config = {
    'experiment_name': 'vertix_hungarian_overfit',
    'device': 'cuda:0',  
    'is_overfit': True,
    'batch_size': 10,
    'resume_ckpt': None,
    'learning_rate': 1e-3,
    'max_epochs': 10000,
    'print_every_n': 1,
    'validate_every_n': 1,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'overfit',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2
}

In [5]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "overfit", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [6]:
dataset.filter_data()

Length of dataset: 91
Filtering data ..


100%|█████████████████████████████████████████| 91/91 [00:00<00:00, 3732.57it/s]

Length of dataset: 10


In [7]:
from training import vertix_train_chamfer

vertix_train.main(config)

ModuleNotFoundError: No module named 'pytorch3d'

In [21]:
from inference.inference_vertix import InferenceHandlerVertixModel

# create a handler for inference using a trained checkpoint
inferer = InferenceHandlerVertixModel('runs/vertix_hungarian_overfit/model_best.ckpt', config["num_vertices"])

In [13]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [13]:
random_sample = random.randint(0,len(dataset))

In [14]:
input_sdf, target_vertices, mask, target_edges, edges_adj = dataset[random_sample]

In [15]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if target_edges[i][j] and target_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

In [16]:
faces = np.concatenate(faces,0)

In [17]:
visualize_pointcloud(target_vertices, point_size=0.01)

Output()

In [22]:
output_pointcloud = inferer.infer_single(input_sdf)

In [23]:
from scipy.optimize import linear_sum_assignment

In [24]:
visualize_pointcloud(output_pointcloud,point_size=0.01)

Output()

In [70]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if target_edges[i][j] and target_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

In [71]:
cost = distance_matrix(output_pointcloud, target_vertices)
                        
vertix_idx, target_idx = linear_sum_assignment(cost)

In [72]:
faces = np.concatenate(faces,0)

In [73]:
visualize_mesh(output_pointcloud[vertix_idx],faces)

Output()